In [1]:
import pandas as pd
import re
from konlpy.tag import Okt, Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from lightgbm import LGBMClassifier

# EDA

In [2]:
train = pd.read_csv('../data/train_data.csv')

In [3]:
# 데이터 확인
train.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


In [4]:
# 결측치 확인
train.isnull().sum()

index        0
title        0
topic_idx    0
dtype: int64

In [5]:
# label 분포 확인
train.topic_idx.value_counts()

topic_idx
4    7629
2    7362
5    6933
6    6751
1    6222
3    5933
0    4824
Name: count, dtype: int64

# 데이터 전처리

In [6]:
okt = Okt()

In [7]:
train['title']

0                  인천→핀란드 항공기 결항…휴가철 여행객 분통
1            실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화
2            이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것
3          NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합
4                 시진핑 트럼프에 중미 무역협상 조속 타결 희망
                        ...                
45649          KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략
45650       1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토
45651           게시판 키움증권 2020 키움 영웅전 실전투자대회
45652                     답변하는 배기동 국립중앙박물관장
45653    2020 한국인터넷기자상 시상식 내달 1일 개최…특별상 김성후
Name: title, Length: 45654, dtype: object

In [8]:
# 조사, 어미, 구두점 제거
def func(text):
    clean = []
    for word in okt.pos(text, stem=True): # 어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']: # 조사, 어미 , 구두점 제외
            clean.append(word[0])
    return " ".join(clean)

train['title'] = train['title'].apply(lambda x : func(x))

In [9]:
train['title']

0                    인천 → 핀란드 항공기 결항 휴가 철 여행객 분통
1               실리콘밸리 넘어서다 구글 15조원 들이다 美 전역 거점 화
2                이란 외무 긴장 완화 해결 책 미국 경제 전쟁 멈추다 것
3        NYT 클린턴 측근 韓 기업 특수 관계 조명 공과 사 맞다 물리다 종합
4                      시진핑 트럼프 중미 무역 협상 조속 타결 희망
                          ...                   
45649               KB 금융 미국 IB 스티펠 제휴 선진국 시장 공략
45650          1 보 서울시 교육청 신종 코로나 확산 개학 연기 휴업 검토
45651          게시판 키우다 증권 2020 키우다 영웅 전 실전 투자 대회
45652                     답변 하다 배 기동 국립 중앙 박물관 장
45653      2020 한국 인터넷 기자 상 시상식 내달 1일 개최 특별상 김성후
Name: title, Length: 45654, dtype: object

In [12]:
# tf-idf를 이용한 vectorization
def split(text):
    tokens_ko = text.split()
    return tokens_ko

tfidf_vect = TfidfVectorizer(tokenizer=split)
tfidf_vect.fit(train['title'])
tfidf_matrix_train = tfidf_vect.transform(train['title'])

C:\Users\Kyeul\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [16]:
# train/valid 데이터셋 나누기
def split_dataset(tfidf, df):
    X_data = tfidf
    y_data = df['topic_idx']

    # stratify = y_data Stratified 기반 분할, train 데이터의 30%를 평가 데이터셋으로 사용.
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42, stratify=y_data)

    return (X_train, X_test, y_train, y_test)

In [17]:
X_train, X_test, y_train, y_test = split_dataset(tfidf_matrix_train, train)

# 모델 학습

In [19]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050377 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 74984
[LightGBM] [Info] Number of data points in the train set: 31957, number of used features: 2872
[LightGBM] [Info] Start training from score -2.247404
[LightGBM] [Info] Start training from score -1.993067
[LightGBM] [Info] Start training from score -1.824812
[LightGBM] [Info] Start training from score -2.040560
[LightGBM] [Info] Start training from score -1.789166
[LightGBM] [Info] Start training from score -1.884794
[LightGBM] [Info] Start training from score -1.911312


LGBMClassifier()

# 모델 평가

In [20]:
pred = lgbm.predict(X_test)
accuracy = accuracy_score(y_test, pred)

print("정확도: ", accuracy)

정확도:  0.8113455501204644


# test 데이터 예측

In [21]:
test = pd.read_csv('../data/test_data.csv')
test

,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
...,...,...
9126,54780,인천 오후 3시35분 대설주의보…눈 3.1cm 쌓여
9127,54781,노래방에서 지인 성추행 외교부 사무관 불구속 입건종합
9128,54782,40년 전 부마항쟁 부산 시위 사진 2점 최초 공개
9129,54783,게시판 아리랑TV 아프리카개발은행 총회 개회식 생중계


In [22]:
test['title'] = test['title'].apply(lambda x : func(x))
tfidf_matrix_test = tfidf_vect.transform(test['title'])

In [23]:
pred = lgbm.predict(tfidf_matrix_test)

In [24]:
pred

array([2, 3, 2, ..., 4, 0, 2], dtype=int64)

# 제출 파일 생성

In [26]:
submission = pd.read_csv('../data/sample_submission.csv')

In [27]:
submission['topic_idx']=pred
submission.head()

,index,topic_idx
0,45654,2
1,45655,3
2,45656,2
3,45657,2
4,45658,3


In [28]:
# save
submission.to_csv('baseline.csv', index=False)